<a href="https://colab.research.google.com/github/aurotripathy/RL-Experiments/blob/main/qwen3_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If this is Colab, ensure you are using a GPU

https://github.com/QwenLM/Qwen-Agent/blob/main/examples/assistant_qwen3.py

In [ ]:
!pip install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
# Or use `pip install -U qwen-agent` for the minimal requirements.
# The optional requirements, specified in double brackets, are:
#   [gui] for Gradio-based GUI support;
#   [rag] for RAG support;
#   [code_interpreter] for Code Interpreter support;
#   [mcp] for MCP support.

In [ ]:
# https://medium.com/@abonia/running-ollama-in-google-colab-free-tier-545609258453
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm

In [ ]:
"""An agent implemented by assistant with qwen3"""
import os  # noqa

from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI
from qwen_agent.utils.output_beautify import typewriter_print

In [ ]:
def init_agent_service():

    llm_cfg = {
        # Use your own model service compatible with OpenAI API by vLLM/SGLang:
        'model': 'Qwen/Qwen3-8B',
        'model_server': 'http://localhost:11434',  # api_base
        'api_key': 'EMPTY',

        'generate_cfg': {
            # When using vLLM/SGLang OAI API, pass the parameter of whether to enable thinking mode in this way
            'extra_body': {
                'chat_template_kwargs': {'enable_thinking': False}
            },

            # Add: When the content is `<think>this is the thought</think>this is the answer`
            # Do not add: When the response has been separated by reasoning_content and content
            # This parameter will affect the parsing strategy of tool call
            # 'thought_in_content': True,
        },
    }
    tools = [
        # {
        #     'mcpServers': {  # You can specify the MCP configuration file
        #         'time': {
        #             'command': 'uvx',
        #             'args': ['mcp-server-time', '--local-timezone=Asia/Shanghai']
        #         },
        #         'fetch': {
        #             'command': 'uvx',
        #             'args': ['mcp-server-fetch']
        #         }
        #     }
        # },
        'code_interpreter',  # Built-in tools
    ]
    bot = Assistant(llm=llm_cfg,
                    function_list=tools,
                    name='Qwen3 Tool-calling Demo',
                    description="I'm a demo using the Qwen3 tool calling. Welcome to add and play with your own tools!")

    return bot

In [ ]:
def test(query: str = 'What time is it?'):
    # Define the agent
    bot = init_agent_service()

    # Chat
    messages = [{'role': 'user', 'content': query}]
    response_plain_text = ''
    for response in bot.run(messages=messages):
        response_plain_text = typewriter_print(response, response_plain_text)


def app_tui():
    # Define the agent
    bot = init_agent_service()

    # Chat
    messages = []
    while True:
        query = input('user question: ')
        messages.append({'role': 'user', 'content': query})
        response = []
        response_plain_text = ''
        for response in bot.run(messages=messages):
            response_plain_text = typewriter_print(response, response_plain_text)
        messages.extend(response)


def app_gui():
    # Define the agent
    bot = init_agent_service()
    chatbot_config = {
        'prompt.suggestions': [
            'What time is it?',
            'https://github.com/orgs/QwenLM/repositories Extract markdown content of this page, then draw a bar chart to display the number of stars.'
        ]
    }
    WebUI(
        bot,
        chatbot_config=chatbot_config,
    ).run()



In [ ]:
if __name__ == '__main__':
    # test()
    # app_tui()
    app_gui()

In [ ]:
!nvidia-smi
